In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The objective is to identify the best sectors, countries,
and a suitable investment type for making investments.

**DATA CLEANING**

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:

# reading data files
# using encoding = "ISO-8859-1" to avoid pandas encoding error
rounds = pd.read_csv("../input/spark-fund-investment-analysis/datasets/rounds2.csv",encoding='iso-8859-1 ')
companies = pd.read_csv("../input/spark-fund-investment-analysis/datasets/companies.txt",sep='\t',encoding='iso-8859-1')

In [ ]:
rounds.head(3)

In [ ]:
companies.head(3)

In [ ]:
rounds.describe()


In [ ]:
companies.describe()

In [ ]:
rounds.shape

In [ ]:
companies.shape

In [ ]:
rounds.info()

some missing values in funding round code and raised amount usd 

In [ ]:
companies.info()

Now what is the unique companies in our data rounds or companies 

In [ ]:
unique_com=rounds['company_permalink'].unique()
unique_com

we can see that we have some companies names are written in lowercase and uppercase 

In [ ]:
unique_com.shape

Now let's make all names in lower case and see th exact number of unique companies 

In [ ]:
rounds['company_permalink']=rounds['company_permalink'].str.lower()

In [ ]:
len(rounds['company_permalink'].unique())


we have 66370 unique companies in rounds data 

we have 66370 unique comapanies in rounds

In [ ]:
companies['permalink']=companies['permalink'].str.lower()

In [ ]:
len(companies['permalink'].unique())

66368 unique companies name in companies data, so rounds file have 2 extra companies 

In [ ]:
# what is the permalink names that are in rounds file but not in 'companies'
list(set(rounds['company_permalink']) - set(companies['permalink']))

Non english character, lets check if this character are present in original file 


this is a problem in encoding of the file . lets detecte which type of encoding of this file 

In [ ]:
#import chardet    
#rawdata = open("../input/spark-fund-investment-analysis/datasets/rounds2.csv", "rb").read()
#result = chardet.detect(rawdata)
#charenc = result['encoding']

In [ ]:
#print(result)

In [ ]:
#rounds_original = pd.read_csv("../input/spark-fund-investment-analysis/datasets/rounds2.csv", encoding="Windows-1254")


In [ ]:
rounds['company_permalink']=rounds.company_permalink.str.encode('utf-8').str.decode('ascii', 'ignore')


In [ ]:
companies['permalink']=companies.permalink.str.encode('utf-8').str.decode('ascii', 'ignore')

In [ ]:
list(set(rounds['company_permalink']) - set(companies['permalink']))

It works well

In [ ]:
print(len(companies.permalink.unique()))
print(len(rounds.company_permalink.unique()))

Now let's deal with missing values 

In [ ]:
companies.isnull().sum()

In [ ]:
rounds.isnull().sum()

In [ ]:
data_tot=pd.merge(companies,rounds,how='inner',left_on="permalink",right_on="company_permalink")

In [ ]:
data_tot.head()

In [ ]:
data_tot.columns

we have a redundant columns 

In [ ]:
# remove reduundant columns
data_tot=data_tot.drop(['company_permalink'], axis=1)

missing values in our data_tot

In [ ]:
data_tot.isnull().sum()

   let' remove all columns which contains a few elements, like funding-round_code,....

In [ ]:
data_tot=data_tot.drop(['city','funding_round_code','founded_at','region','state_code'],axis=1)

for our analysis homepage_url is useless we can drop it 


In [ ]:
data_tot=data_tot.drop(['homepage_url'],axis=1)

In [ ]:
data_tot.head()

In [ ]:
# summing up the missing values and displaying fraction of NaNs
round(100*(data_tot.isnull().sum()/len(data_tot.index)), 2)

 raised_amount_usd is an important columns for our analysis, so let's look carefully at
it 

In [ ]:
r=data_tot['raised_amount_usd']

In [ ]:
r.describe()

we have a lot of missing values, and we can see the important difference between mean, std, mode , it's more suitable to delete all this Nan values

In [ ]:
# removing NaNs in raised_amount_usd
data_tot = data_tot[~np.isnan(data_tot['raised_amount_usd'])]

In [ ]:
round(100*(data_tot.isnull().sum()/len(data_tot.index)), 2)

Now let's pass to country-code

In [ ]:
data_tot['country_code']

In [ ]:
country_codes = data_tot['country_code'].astype('category')

# displaying frequencies of each category
country_codes.value_counts()

we see clearly that most of the companies are in USA, we can fill our missing values by USA or remove them since they are few missing values

In [ ]:
round(100*(data_tot['country_code'].value_counts()/len(data_tot.index)), 2)

In [ ]:
# removing NaNs in country_code
data_tot = data_tot[~pd.isnull(data_tot['country_code'])]

In [ ]:
round(100*(data_tot.isnull().sum()/len(data_tot.index)), 2)

In [ ]:
# removing NaNs in category_list
data_tot = data_tot[~pd.isnull(data_tot['category_list'])]

In [ ]:
round(100*(data_tot.isnull().sum()/len(data_tot.index)), 2)

Now we treated all missing value

In [ ]:
data_tot.info()

**Data Analysis**

let's have a look at the distribution of raised_amount_usd to get a sense of the distribution of data.

In [ ]:
# distribution of raised_amount_usd
sns.boxplot(y=data_tot['raised_amount_usd'])
plt.yscale('log')
plt.show()

In [ ]:
# the summary metrics of raised_amount_usd
data_tot['raised_amount_usd'].describe()

Note that there's a significant difference between the mean and the median

In [ ]:
data_tot['funding_round_type'].unique()

In [ ]:
round(100*(data_tot['funding_round_type'].value_counts()/len(data_tot.index)), 2)

In [ ]:
# first, let's filter the df so it only contains the four specified funding types
data_tot = data_tot[(data_tot.funding_round_type == "venture") | 
        (data_tot.funding_round_type == "angel") | 
        (data_tot.funding_round_type == "seed") | 
        (data_tot.funding_round_type == "private_equity") ]

In [ ]:
# comparing summary stats across categories

sns.boxplot(x='funding_round_type', y='raised_amount_usd', data=data_tot)
plt.yscale('log')
plt.show()

In [ ]:
# compare the mean and median values across categories
data_tot.pivot_table(values='raised_amount_usd', columns='funding_round_type', aggfunc=[np.median, np.mean])

Let's choose median, since there are quite a few extreme values pulling the mean up towards them - but they are not the most 'representative' values.

In [ ]:
data_tot.groupby('funding_round_type')['raised_amount_usd'].median().sort_values(ascending=False)

The median investment amount for type 'private_equity' is approx. USD 20m, which is beyond Teclov' range of 5-15m. The median of 'venture' type is about USD 5m, which is suitable for them. The average amounts of angel and seed types are lower than their range.
so the 'venture' is the most suitable for Teclov company 

**Country Analysis**

In [ ]:
# filter the df for private equity type investments
data_tot = data_tot[data_tot.funding_round_type=="venture"]

# group by country codes and compare the total funding amounts
data_tot.groupby('country_code')['raised_amount_usd'].sum().sort_values(ascending=False)


so we can see clearly that USA is the most suitable country and Chin and IND , they are the top english speaking country 

In [ ]:
data_tot.groupby('country_code')['raised_amount_usd'].sum().sort_values(ascending=False)[:5]

The top 5 countries whch have the best funding 

In [ ]:
# filtering for the top three countries
data_tot = data_tot[(data_tot.country_code=='USA') | (data_tot.country_code=='GBR') | (data_tot.country_code=='IND')]
data_tot.head()

In [ ]:
data_tot.info()

In [ ]:
# boxplot to see distributions of funding amount across countries
plt.figure(figsize=(10, 10))
sns.boxplot(x='country_code', y='raised_amount_usd', data=data_tot)
plt.yscale('log')
plt.show()

In [ ]:
# extracting the main category
data_tot.loc[:, 'main_category'] =data_tot['category_list'].apply(lambda x: x.split("|")[0])
data_tot.head()

In [ ]:
# read mapping file
mapping = pd.read_csv("../input/spark-fund-investment-analysis/datasets/mapping.csv", sep=",")
mapping.head()

In [ ]:
# missing values in mapping file
mapping.isnull().sum()

In [ ]:
# remove the row with missing values
mapping = mapping[~pd.isnull(mapping['category_list'])]
mapping.isnull().sum()

In [ ]:
# converting common columns to lowercase
mapping['category_list'] = mapping['category_list'].str.lower()
data_tot['main_category'] = data_tot['main_category'].str.lower()

In [ ]:
# look at heads
print(mapping.head())

In [ ]:
print(data_tot.head())

In [ ]:
mapping['category_list']

In [ ]:
# values in main_category column in df which are not in the category_list column in mapping file
data_tot[~data_tot['main_category'].isin(mapping['category_list'])]

In [ ]:
# values in the category_list column which are not in main_category column 
mapping[~mapping['category_list'].isin(data_tot['main_category'])]

In [ ]:
# replacing '0' with 'na'
mapping['category_list'] = mapping['category_list'].apply(lambda x: x.replace('0', 'na'))
print(mapping['category_list'])

In [ ]:
# merge the dfs
data_tot = pd.merge(data_tot, mapping, how='inner', left_on='main_category', right_on='category_list')
data_tot.head()

In [ ]:
# look at the column types and names
data_tot.info()

In [ ]:
help(pd.melt)

In [ ]:
# store the value and id variables in two separate arrays

# store the value variables in one Series
value_vars = data_tot.columns[9:18]

# take the setdiff() to get the rest of the variables
id_vars = np.setdiff1d(data_tot.columns, value_vars)

print(value_vars, "\n")
print(id_vars)

In [ ]:
# convert into long
long_df = pd.melt(data_tot, 
        id_vars=list(id_vars), 
        value_vars=list(value_vars))

long_df.head()

In [ ]:
# remove rows having value=0
long_df = long_df[long_df['value']==1]
long_df = long_df.drop('value', axis=1)

In [ ]:
# look at the new df
long_df.head()
len(long_df)

In [ ]:
# renaming the 'variable' column
long_df = long_df.rename(columns={'variable': 'sector'})

In [ ]:
# info
long_df.info()

In [ ]:
# summarising the sector-wise number and sum of venture investments across three countries

# first, let's also filter for investment range between 5 and 15m
data_tot = long_df[(long_df['raised_amount_usd'] >= 5000000) & (long_df['raised_amount_usd'] <= 15000000)]


In [ ]:
# groupby country, sector and compute the count and sum
data_tot.groupby(['country_code', 'sector']).raised_amount_usd.agg(['count', 'sum'])

In [ ]:
# plotting sector-wise count and sum of investments in the three countries
plt.figure(figsize=(16, 14))

plt.subplot(2, 1, 1)
p = sns.barplot(x='sector', y='raised_amount_usd', hue='country_code', data=data_tot, estimator=np.sum)
p.set_xticklabels(p.get_xticklabels(),rotation=30)
plt.title('Total Invested Amount (USD)')

plt.subplot(2, 1, 2)
q = sns.countplot(x='sector', hue='country_code', data=data_tot)
q.set_xticklabels(q.get_xticklabels(),rotation=30)
plt.title('Number of Investments')


plt.show()